In [2]:
%pip install -r ../requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 373.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 216.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 200.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 312.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 245.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 266.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 274.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import pandas as pd
from minio import Minio

In [8]:
import digitalhub as dh
import digitalhub_data as dh_data

# Example of creating a DH project

In [9]:
project = dh.get_or_create_project(
    name="overtourism"
)

In [6]:
contamezzi_di = project.new_dataitem(
                    name="contamezzi",
                    kind="table",
                    path="s3://datalake/projects/tourism/molveno/contamezzi.parquet")


In [8]:
obj = dh_data.get_dataitem(
                    project="overtourism",
                    entity_id="a634372f-a792-4bbd-8052-069932c4c0e2")
obj

{'project': 'overtourism', 'name': 'contamezzi', 'id': 'a634372f-a792-4bbd-8052-069932c4c0e2', 'kind': 'table', 'key': 'store://overtourism/dataitems/table/contamezzi:a634372f-a792-4bbd-8052-069932c4c0e2', 'metadata': {'project': 'overtourism', 'name': 'contamezzi', 'version': 'a634372f-a792-4bbd-8052-069932c4c0e2', 'created': '2024-09-27T14:54:37.329Z', 'updated': '2024-09-27T14:54:37.329Z', 'created_by': 'u_328e892e9fb74d0baeb9c616bc13475b', 'updated_by': 'u_328e892e9fb74d0baeb9c616bc13475b', 'embedded': True}, 'spec': {'path': 's3://datalake/projects/tourism/molveno/contamezzi.parquet'}, 'status': {'state': 'CREATED'}, 'user': 'u_328e892e9fb74d0baeb9c616bc13475b'}

In [9]:
obj.as_df()

,data,sensore,direzione,comune,cl1,cl2,cl3,cl4,cl5,cl6,cl7,nonrilevato
0,2023-06-29 09:00:00,M40,1,MOLVENO,76,16,1,1,2,0,0,1
1,2023-06-29 09:00:00,M40,2,MOLVENO,53,12,4,0,2,0,0,3
2,2023-06-29 10:00:00,M40,1,MOLVENO,166,44,5,3,4,1,0,2
3,2023-06-29 10:00:00,M40,2,MOLVENO,116,31,5,6,1,0,0,6
4,2023-06-29 11:00:00,M40,1,MOLVENO,147,38,4,8,3,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...
13821,2023-10-04 21:00:00,M47,2,MOLVENO,11,0,0,0,0,0,0,0
13822,2023-10-04 22:00:00,M47,1,MOLVENO,1,2,0,0,0,0,0,0
13823,2023-10-04 22:00:00,M47,2,MOLVENO,3,0,0,0,0,0,0,0
13824,2023-10-04 23:00:00,M47,1,MOLVENO,4,1,0,0,0,0,0,0


In [13]:
minio_client = Minio(
    "minio-api.digitalhub-test.smartcommunitylab.it",
    access_key="AK",
    secret_key="SK"
)

# Download overtourism datasets from the minio bucket

In [ ]:
objects = minio_client.list_objects("datalake", prefix="projects/tourism", recursive=True)
for obj in objects:
    # download dataset locally
    minio_client.fget_object(obj.bucket_name, obj.object_name, f"datasets/{obj.object_name[len('projects/tourism/'):]}")
    file = obj.object_name[len('projects/tourism/'):]
    print(file)
    #if True: #file.startswith("ristorazione"):
    if not file.startswith("_origin"): 
        print(file[file.index("/")+1:])
        df = pd.read_parquet(f"datasets/{obj.object_name[len('projects/tourism/'):]}")
        
        # upload dataset into the platform project
        project.new_dataitem(
                    name=file[file.index("/")+1:],
                    kind="table",
                    path=f"s3://datalake/projects/tourism/{file[file.index('/')+1:]}")
        
